<a href="https://colab.research.google.com/github/ritwiks9635/My_priject_file_4/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Fake News means incorporating information that leads people to the wrong paths. It can have real-world adverse effects that aim to intentionally deceive, gain attention, manipulate public opinion, or damage reputation. It is necessary to detect fake news mainly for media outlets to have the ability to attract viewers to their website to generate online advertising revenue.

Import modules

In [ ]:
import numpy as np
import pandas as pd

  
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
  
import pprint

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()

In [ ]:
# load dataset

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/news.csv")
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


Preprocessing Dataset

In [ ]:
# The dataset contains one unnamed column. So we drop that column from the dataset.

In [ ]:
data = data.drop(["Unnamed: 0"], axis = 1)
data.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


Data Encoding

In [ ]:
#It converts the categorical column into numerical values.

In [ ]:
le = preprocessing.LabelEncoder()
data["label"] = le.fit_transform(data["label"])
data.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


In [ ]:
# required some variables 
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

Tokenization

In [ ]:
# This process divides a large piece of continuous text into distinct units or tokens basically

In [ ]:
title = []
text = []
label = []
for x in range(training_size):
  title.append(data["title"][x])
  text.append(data["text"][x])
  label.append(data["label"][x])

In [ ]:
len(title)

3000

In [ ]:
# Applying Tokenization

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(title)
word_index = tokenizer.word_index
vocab_size = len(word_index)
sequence = tokenizer.texts_to_sequences(title)
pad_sequences = pad_sequences(sequence, padding = padding_type, truncating = trunc_type)
split = int(test_portion * training_size)
training_sequence = pad_sequences[split : training_size]
test_sequence = pad_sequences[0 : split]
training_labels = label[split : training_size]
test_labels = label[0 : split]

In [ ]:
len(training_sequence), len(test_sequence)

(2700, 300)

Generating Word Embedding

In [ ]:
# It allows words with similar meanings 
# to have a similar representation.
# Here each individual word is
# represented as real-valued vectors
# in a predefined vector space.

In [ ]:
embedding_in = {}
with open("/content/drive/MyDrive/Copy of glove.6B.50d.txt") as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embedding_in[word] = coefs 

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_in.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

Creating Model Architecture

In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim,
                              input_length=max_length,
                              weights=[embeddings_matrix],
                              trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54, 50)            377600    
                                                                 
 dropout (Dropout)           (None, 54, 50)            0         
                                                                 
 conv1d (Conv1D)             (None, 50, 64)            16064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                        

Train the Model

In [ ]:
training_pad = np.array(training_sequence)
training_labels = np.array(training_labels)
test_pad = np.array(test_sequence)
test_labels = np.array(test_labels)

In [ ]:
model.fit(training_pad, training_labels, validation_data = (test_pad, test_labels), epochs = 50, verbose=2)

Train on 2700 samples, validate on 300 samples
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 3s - loss: 0.6417 - accuracy: 0.6196 - val_loss: 0.5569 - val_accuracy: 0.6967 - 3s/epoch - 1ms/sample
Epoch 2/50
2700/2700 - 2s - loss: 0.5790 - accuracy: 0.6956 - val_loss: 0.5221 - val_accuracy: 0.7133 - 2s/epoch - 686us/sample
Epoch 3/50
2700/2700 - 2s - loss: 0.5388 - accuracy: 0.7330 - val_loss: 0.4937 - val_accuracy: 0.7433 - 2s/epoch - 778us/sample
Epoch 4/50
2700/2700 - 3s - loss: 0.5001 - accuracy: 0.7456 - val_loss: 0.5193 - val_accuracy: 0.7267 - 3s/epoch - 1ms/sample
Epoch 5/50
2700/2700 - 2s - loss: 0.4572 - accuracy: 0.7844 - val_loss: 0.4550 - val_accuracy: 0.7733 - 2s/epoch - 725us/sample
Epoch 6/50
2700/2700 - 2s - loss: 0.3931 - accuracy: 0.8219 - val_loss: 0.4845 - val_accuracy: 0.7533 - 2s/epoch - 674us/sample
Epoch 7/50
2700/2700 - 2s - loss: 0.3807 - accuracy: 0.8404 - val_loss: 0.4829 - val_accuracy: 0.7633 - 2s/epoch - 668us/sample
Epoch 8/50
2700/2700 - 2s - loss: 0.3152 - accuracy: 0.8678 - val_loss: 0.4875 - val_accuracy: 0.7767 - 2s/epoch - 667u

Model Prediction

In [ ]:
#Now we will try to test the model by using some news text by predicting whether it is true or false.

In [ ]:
# sample text to check if fake or not
X = "Bernie supporters on Twitter erupt in anger against the DNC: 'We tried to warn you!'"
  
# detection
sequences = tokenizer.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
                          padding=padding_type, 
                          truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
    print("This news is True")
else:
    print("This news is false")

This news is false
